### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
#import cpi #library for inflation-data
df = pd.read_csv('C:/Users/admin1/Documents/GitHub/ds22_project/data/movies.csv')

### Functions

#### 1.1 --- check_non_numeric_values

In [ ]:
def check_non_numeric_values(df, column):
    """Function takes in dataset and column. No kreturn, Printing out found non numeric values in the column."""

    # convert column to numeric data type
    numeric_col = pd.to_numeric(df[column], errors='coerce')

    # get the non-numeric values and their counts
    non_numeric_values = df[column][numeric_col.isna()].value_counts()

    # check if there are any non-numeric values
    if non_numeric_values.empty:
        print("No non numeric values in that column.")
    else:
        # create a table with non-numeric values and their counts
        non_numeric_table = pd.DataFrame({'Non-Numeric Value': non_numeric_values.index,
                                          'Count': non_numeric_values.values})

        # display the table
        print(non_numeric_table)

#### 1.2 --- get_mean_median_for_column

In [ ]:
def get_mean_median_for_column(df, col_name):
    '''
    This function takes in a pandas dataframe and the name of a column in the dataframe,
    and returns the mean and median of the numeric values in the column that are not equal to 0.
    Non-numeric values are converted to 0 before calculating the mean and median.
    
    Parameters:
    - df: pandas dataframe
    - col_name: str, name of column to be processed
    
    Returns:
    - tuple of two floats: mean and median of numeric values in the column that are not equal to 0
    '''
    # Convert non-numeric values to 0
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce').fillna(0)
    
    # Get the non-zero numeric values in the column
    non_zero_vals = df[col_name][df[col_name] != 0]
    
    # Calculate the mean and median of the non-zero values
    col_mean = non_zero_vals.mean()
    col_median = non_zero_vals.median()
    
    return col_mean, col_median


#### 1.3 --- replace_missing_values

In [ ]:
def replace_missing_values(df, col_name, stat='mean'):
    '''
    This function takes in a pandas dataframe and the name of a column in the dataframe.
    It drops all rows where the value of the column is 0, and replaces those values with either 
    the mean or median of the rest of the values in the column, as specified by the user.
    It also replaces any NaN values in the column with the same statistic as the missing values.
    
    Parameters:
    - df: pandas dataframe
    - col_name: str, name of column to be processed
    - stat: str, either 'mean' or 'median', determines which statistic to use
    
    Returns:
    - df: pandas dataframe with modified column
    '''
    # Calculate the selected statistic of the non-zero/non-NaN values in the column
    if stat == 'mean':
        stat_val = np.nanmean(df[df[col_name].notnull() & (df[col_name] != 0)][col_name])
    elif stat == 'median':
        stat_val = np.nanmedian(df[df[col_name].notnull() & (df[col_name] != 0)][col_name])
    else:
        raise ValueError("stat must be either 'mean' or 'median'")
    
    # Replace the missing values (0 or NaN) with the selected statistic
    df.loc[(df[col_name] == 0) | (df[col_name].isnull()), col_name] = stat_val
    
    return df

#### 2.1 --- convert_to_usd

In [ ]:
def convert_to_usd(amount):
    amount.replace(' ', '')
    amount.replace('\xa0', '')
    if amount.startswith('$'):
        amount = amount.strip('$').replace(',', '')   # must remove commas
        return float(amount)   # convert str into float
    elif amount.startswith('€'):
        # Exchange rate for EUR to USD
        amount = amount.strip('€').replace(',', '')
        return float(amount) * 1.06 
    elif amount.startswith('¥'):
        # Exchange rate for YEN to USD
        amount = amount.strip('¥').replace(',', '')
        return float(amount) * 0.0075
    elif amount.startswith('₹'):
        # Exchange rate for RPL to USD
        amount = amount.strip('₹').replace(',', '')
        return float(amount) * 0.012 
    elif amount.startswith('SEK'):
        # Exchange rate for SEK to USD
        amount = amount.strip('SEK').replace(',', '')
        return float(amount) * 0.094
    elif amount.startswith('DKK'):
        # Exchange rate for RPL to USD
        amount = amount.strip('DKK').replace(',', '')
        return float(amount) * 0.14
    elif amount.startswith('£'):
        # Exchange rate for RPL to USD
        amount = amount.strip('£').replace(',', '')
        return float(amount) * 1.21  
    else:
        return None

#### 2.2 --- adjust_for_inflation

In [ ]:
def adjust_for_inflation(df, column_name, year_column, new_column, drop_original=True):
    data = {
        "1990": 5.398,
        "1991": 4.235,
        "1992": 3.0288,
        "1993": 2.9517,
        "1994": 2.6074,
        "1995": 2.8054,
        "1996": 2.9312,
        "1997": 2.3377,
        "1998": 1.5523,
        "1999": 2.188,
        "2000": 3.3769,
        "2001": 2.8262,
        "2002": 1.586,
        "2003": 2.2701,
        "2004": 2.6772,
        "2005": 3.3927,
        "2006": 3.2259,
        "2007": 2.8527,
        "2008": 3.8391,
        "2009": -0.3555,
        "2010": 1.64,
        "2011": 3.1568,
        "2012": 2.0693,
        "2013": 1.4648,
        "2014": 1.6222,
        "2015": 0.1186,
        "2016": 1.2616,
        "2017": 2.1301,
        "2018": 2.4426,
        "2019": 1.8122,
        "2020": 1.2336,
        "2021": 4.6979
    }
    
    # Create a new column in the DataFrame to store the adjusted values
    df[new_column] = 0

    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the year from the row
        year = row[year_column]
        # Skip the row if the value in the specified column is NaN
        if pd.isna(row[column_name]):
            continue
        # Get the inflation rate for each year from the dictionary
        inflation_rates = [data[str(yr)] for yr in range(year, 2022)]
        # Calculate the total inflation factor by multiplying the inflation rates together
        total_inflation_factor = 1
        for rate in inflation_rates:
            total_inflation_factor *= 1 + (rate / 100)
        # Get the value from the specified column
        value = row[column_name]
        # Adjust the value for inflation using the total inflation factor
        adjusted_value = value * total_inflation_factor
        # Round the result to two decimal places and store it in the new column
        df.at[index, new_column] = round(adjusted_value, 2)

    # Drop the original column if specified
    if drop_original:
        df = df.drop(columns=[column_name])

    # Return the DataFrame with the adjusted values
    return df


#### 3.1 --- adjust_for_inflation

In [ ]:
def one_hot_encoding_column(dataset, column, separator=", ", prefix=""):
    """
    Performs one-hot encoding on the specified column of the given dataset.
    dataset: The dataset to be processed.
    column: The name of the column to be one-hot encoded.
    separator: The separator used in the values of the specified column. Defaults to ",".
    prefix: Optional string to be added in front of each new column name. Defaults to "".
    returns: the new dataset with the specified column one-hot encoded.
    """

    # 1. Creating a list with all the values mentioned in the dataset
    value_list = [values.split(separator) for values in dataset[column]]

    # 2. Creating a set with value categories
    unique_v = {value for values in value_list for value in values}

    # 3. Performing one-hot encoding using get_dummies method
    value_subtable = pd.get_dummies(dataset[column].str.split(separator, expand=True).stack()).reset_index(level=1, drop=True)
    value_subtable = value_subtable.groupby(value_subtable.index).sum()

    # 4. Adding the prefix to the column names
    if prefix:
        value_subtable.columns = [prefix + str(col) for col in value_subtable.columns]

    # 5. Merging the subtable with the main dataset
    dataset_processed = pd.merge(dataset, value_subtable, left_index=True, right_index=True, how='left')
    dataset_processed.drop(columns=[column], inplace=True)

    # 6. Returning the new dataset
    return dataset_processed

#### 3.2 --- one_hot_coding_binary

In [ ]:
def one_hot_coding_binary(dataset, original_column, prefix, file_column, file_location, separator=", ", num_categories=1, drop_original=True):
    if num_categories not in range(1,5):
        raise ValueError("num_categories must be between 1 and 4")

    for i in range(1, num_categories+1):
        dataset[f"{prefix}_no_{i}"] = dataset[original_column].str.split(separator, expand=True)[i-1]

    dataset_categories = pd.read_csv(file_location)

    for i in range(1, num_categories+1):
        replace = dataset[f"{prefix}_no_{i}"].isin(dataset_categories[file_column])
        dataset[f"{prefix}_no_{i}_binary"] = replace.astype(int)

    if drop_original:
        dataset.drop(columns=[original_column], inplace=True)

    if num_categories == 1:
        dataset.drop(columns=[f"{prefix}_no_1"], inplace=True)
        dataset.rename(columns={f"{prefix}_no_1_binary": f"{prefix}"}, inplace=True)
    else:
        for i in range(1, num_categories+1):
            dataset.drop(columns=[f"{prefix}_no_{i}"], inplace=True)
            dataset.rename(columns={f"{prefix}_no_{i}_binary": f"{prefix}_{i}"}, inplace=True)

        if num_categories == 5:
            dataset.rename(columns={f"{prefix}_all_binary": f"{prefix}_all"}, inplace=True)

    return dataset


# Dataprocessing

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df.describe()

### 1.Title

#### 1.1Unqiue/ Dupclicate values

In [ ]:
num_unique_values = df['Title'].nunique()

In [ ]:
print(num_unique_values)

My first instict was to expect 2000 unique values for titles of movies.
So I thought to just remove them from the dataset since I expect them to be duplicate data.
But.

In [ ]:
duplicates = df[df.duplicated(['Title'], keep=False)].sort_values(by=['Title'])

duplicates

Looking at the duplicate movie titles we can quickly see that its totally diffrent movies that only share the title name and nothing else therefore no duplicate data and we can keep it.
I do expect that we drop this column since we cant make in to numeric.
Well, you could keep it and count the length of the title but I would say that shouldnt have an effect on the model and just be in the way.

### 2.Rating

Rating of the movie, the user rates 0-10 and this it the average of that voting with 1 decimal. This is our y, the data that we want to predict. 

#### 2.1 Missing data
According to the info-function, there is one movie that doesn´t have a value. We cant do more than just drop that one from the dataset.



In [ ]:
df = df.dropna(subset=['Rating'])


#A quick check to see that the row was removed from the dataset
df.info()


Since this is what our model is gonna predict, I dont want to do more here. The type is float64, which tells us that all values is numeric and can be decimal and since imdb is ratings from 0-10 with decimals this seems correct. One thing to consider would be to 10x all the values in ratings and convert the column to an int since that will be quicker to work with.
Actually wann quick check that all the values is between 0 and 10.

#### 2.2 Check the data is within expected range

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Rating']
count = column_values.between(0, 10).sum()

In [ ]:
print(count)

Our orginial dataset consistet of 2000 rows and we dropped one with missing values so 1999 was what we expected and hoped for. 


### 3.Year

#### 3.1 Check the data is within expected range

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Year']
count = column_values.between(2002, 2023).sum()

In [ ]:
print(count)

The type of the column is int which make sense aswell.

### 4.Month

#### 4.1 Unqiue/ Dupclicate values



In [ ]:
column_values = df['Month'].unique()
print(column_values)

Seeing two values that I didnt expect,  2014 and 2018. Start with checking number of times they appear.

In [ ]:
# Count the number of times a specific string value occurs in a column
count = df['Month'].value_counts()['2014']
print(count)

In [ ]:
# Count the number of times a specific string value occurs in a column
count = df['Month'].value_counts()['2008']
print(count)

Since they just occurs one time each in the Month-column I suggest we drop them since 2/ 1999 rows one impact our size of dataset especcially much and not seeing it beeing the worth the time to save

In [ ]:
df = df.drop(index=df.loc[df['Month'] == '2014'].index)
df = df.drop(index=df.loc[df['Month'] == '2008'].index)

In [ ]:
#checking the rows have been dropped
df.info()

In [ ]:
column_values = df['Month'].unique()
print(column_values)

#### 4.2 Converting non-numeric to numeric values



In [ ]:
# Define a dictionary to map months to integers
month_to_int = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

# Apply the map() method to convert the values
df['Month'] = df['Month'].map(month_to_int)

# Convert the type of the column to int
df['Month'] = df['Month'].astype(int)

In [ ]:
#checking the month-values has been rplaced by 1-12 and the column converted to int
df.info()

In [ ]:
column_values = df['Month'].unique()
column_values.sort()
print(column_values)

### 5.Certificate

In [ ]:
df.info()

In [ ]:
### 5.1 Unique values

In [ ]:
# Get all unique values of a column
unique_values = df['Certificate'].unique()

In [ ]:
print(unique_values)

In [ ]:
# Get the count of each unique value in the column, including missing values
value_counts = df['Certificate'].value_counts(dropna=False)

# Print the value counts
print(value_counts)

# Calculate the sum of the counts and print the total
total = value_counts.sum()
print(f'Total: {total}')

A couple of things. Tree columns stand out.
Not Rated     61   --- probably takes a while from a movie is realsed until it gets rated. Make sense that this mostly consists of movies from the past year
NaN           32   --- not sure how to replace
Unrated        6   --- not sure how to replace

At this point Im not sure how to replace the missing values and w/ ca 100 rows w/ diffrent kind of missing values its a bit much to drop them.
My conclusion is to drop the whole column.


### 6.Runtime

#### 6.1 Check non numeric

In [ ]:
#Checking non numeric values
check_non_numeric_values(df, "Runtime")

In [ ]:
df = df.drop(index=df.loc[df['Runtime'] == 'Unknown'].index)

In [ ]:
#check that the row has been dropped
df.info()

In [ ]:
#converting the type of the column to int
df['Runtime'] = df['Runtime'].astype(int)

In [ ]:
#checking the type has been changed
df.info()

check for outliers, since movies should maybe have a range from 30-250min isch.
So if i found values >30 or over 300 min I can assumme that something is wrong and drop them

In [ ]:
# Count how many values are between 0 and 10 (inclusive)
column_values = df['Runtime']
count = column_values.between(30, 300).sum()

In [ ]:
print(count)

Seems all remaining rows has a value between 30-300min which is good news.

### 7.Directors

In [ ]:
df.info()

In [ ]:
#calling the function
df = one_hot_coding_binary(df, "Directors", "top_50_director", "Name", "C:/Users/admin1/Documents/GitHub/ds22_project/data/top_50_directors.csv", num_categories=4, drop_original=True)


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#checking sum of each row
director_one = df['top_50_director_1'].sum()
director_two = df['top_50_director_2'].sum()
director_three = df['top_50_director_3'].sum()
director_four = df['top_50_director_4'].sum()


df_directors = pd.DataFrame({'director_one': [director_one], 'director_two': [director_two], 'director_three': [director_three], 'director_four': [director_four]})

df_directors.head()


### 8.Stars

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
#calling the function
df = one_hot_coding_binary(df, "Stars", "top_1000_Stars", "Name", "C:/Users/admin1/Documents/GitHub/ds22_project/data/top_1000_actors.csv", num_categories=4, drop_original=True)


In [ ]:
df.head(2)

In [ ]:
df.info()

### 9.Genre

In [ ]:
df.info()

In [ ]:
#one hot encoding the column Genre
df = one_hot_encoding_column(df, "Genre", separator=", ", prefix = "")

In [ ]:
#checking new dataset
df.info()
df.head(2)

### 10.Filmning_location

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
# assume that `df` is your pandas DataFrame object
column_values = df['Filming_location'].value_counts().sort_values(ascending=False)
print(column_values)

In [ ]:
#Seeing 75 movies with Unknown filming_location. How can we replace them? and seeing 97 unique filming locations.
#We we´re discussing if movies mostly are beeing done w/ green screen.
#maybe remove the whole column?

#one hot encoding the column Filming_location
#df = one_hot_encoding_column(df, "Filming_location", separator=", ", prefix = "")

### 11.Budget

In [ ]:
df.head(4)

In [ ]:
#1.1 convert to USD and strip of non numeric characters
df['Budget'] = df['Budget'].apply(convert_to_usd)

In [ ]:
#check the change
df.head(4)

In [ ]:
df.info()

In [ ]:
#1.2 --- calc with inflation
# Call the function to get the inflation-adjusted values for the "value" column
df = adjust_for_inflation(df, "Budget", "Year", "Budget_inf", drop_original=True)

In [ ]:
#checking the change
df.head(4)

In [ ]:
df.info()

In [ ]:
#1.3 check the mean and median of the column /value of 0 is not a part of the calculation
get_mean_median_for_column(df, "Budget_inf")

In [ ]:
#1.4 replace missing values w/ mean or median
df = replace_missing_values(df, "Budget_inf", stat='median')

In [ ]:
#check the changes

In [ ]:
df.head(4)

In [ ]:
df.info()

### 12.Income

In [ ]:
df.head(4)

In [ ]:
#1.1 convert to USD and strip of non numeric characters
df['Income'] = df['Income'].apply(convert_to_usd)

In [ ]:
df.head(4)

In [ ]:
#1.2 --- calc with inflation
# Call the function to get the inflation-adjusted values for the "value" column
df = adjust_for_inflation(df, "Income", "Year", "Income_inf", drop_original=True)

In [ ]:
df.head(4)

In [ ]:
#1.3 check the mean and median of the column
get_mean_median_for_column(df, "Income_inf")

In [ ]:
df.head(4)

In [ ]:
#1.4 replace missing values w/ mean or median
df = replace_missing_values(df, "Income_inf", stat='mean')

### 13.Country_of_origin

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
#one hot encoding the column Genre
df = one_hot_encoding_column(df, "Country_of_origin", separator=", ", prefix = "")

In [ ]:
df.info()

In [ ]:
df.head(2)